# ディレクトリ内の画像ファイルを元にディープラーニング

In [1]:
import os
import numpy as np
import cv2

TRAIN_DIR = './data/flickr_logos_27_dataset_cropped_augmented_images/'

NUM_CLASSES = 27 # 分類するクラス数
IMG_SIZE = 32 # 画像の1辺の長さ

# 画像のあるディレクトリ
train_img_dirs = ['Adidas','Apple','BMW','Citroen','Cocacola','DHL','Fedex','Ferrari','Ford','Google','Heineken','HP','Intel','McDonalds','Mini','Nbc','Nike','Pepsi','Porsche','Puma','RedBull','Sprite','Starbucks','Texaco','Unicef','Vodafone','Yahoo']


In [2]:
def load_data():

    # 学習画像データ
    images = []
    # 学習データのラベル
    labels = []

    for i, d in enumerate(train_img_dirs):
        # ./data/以下の各ディレクトリ内のファイル名取得
        files = os.listdir(TRAIN_DIR + d)
        for f in files:
            # 画像読み込み
            img = cv2.imread(TRAIN_DIR + d + '/' + f)
            # 1辺がIMG_SIZEの正方形にリサイズ
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            # 1列にして
            img = img.flatten().astype(np.float32)/255.0
            images.append(img)

            # one_hot_vectorを作りラベルとして追加
            tmp = np.zeros(NUM_CLASSES)
            tmp[i] = 1
            labels.append(tmp)

    # numpy配列に変換
    images = np.asarray(images)
    labels = np.asarray(labels)
    
    return images, labels

#### モデルの作成

In [3]:
def build_model():
    # モデルの作成
    model = Sequential()
    #model.add(Dense(512, input_shape=(784,)))
    model.add(Dense(512, input_shape=(3072,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
 
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
 
    model.add(Dense(10))
    model.add(Activation('softmax'))
 
    # 損失関数の定義
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy'])
    
    return model

#### plot用関数

In [4]:
def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['acc'],"o-",label="accuracy")
    plt.plot(history.history['val_acc'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()
 
    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.show()

### 機械学習開始

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam 
from keras.utils import np_utils
 
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
X, y = load_data()
print('images',X)
print('labels',y)

### テストデータと教師データに分割

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

### 機械学習の条件定義

In [ ]:
nb_epoch = 50 #学習させる回数. pytyonのnb_epochとはrangeの繰り返しのこと
batch_size = 128 #無作為に128画像取得。128という数字に理論的な根拠は考慮していない

In [ ]:
# データで訓練
model = build_model()
history = model.fit(X_train, y_train, 
    nb_epoch=nb_epoch, 
    batch_size=batch_size,
    validation_data=(X_test, y_test)
)

In [ ]:
#学習モデルの保存
json_string = model.to_json()

#モデルのファイル名　拡張子.json
open('mnist.json', 'w').write(json_string)
#重みファイルの保存 拡張子がhdf5
model.save_weights('out/mnist.hdf5')
 
# モデルの評価を行う
score = model.evaluate(X_test, y_test, verbose=1)
 
print('loss=', score[0])
print('accuracy=', score[1])
    
# modelに学習させた時の変化の様子をplot
plot_history(history)